In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams.update({'font.size': 14})

In [ ]:
#df = pd.read_csv('/home/jvdzwaan/data/dncvu/word_mapping_gs_ocr.csv', index_col=0, encoding='utf-8')
#df = pd.read_csv('/home/jvdzwaan/data/dncvu/word_mapping_gs_1x256-bidirect.csv', encoding='utf-8')
#df = pd.read_csv('/home/jvdzwaan/data/dncvu/word_mapping_gs_1x256.csv', encoding='utf-8')
df2 = pd.read_csv('/home/jvdzwaan/data/dncvu/word_mapping_gs_ocr-lowercase-test-files.csv', encoding='utf-8')
df = pd.read_csv('/home/jvdzwaan/data/dncvu/word_mapping_gs_256-seq2seq-lowercase.csv', encoding='utf-8')
#df = pd.read_csv('/home/jvdzwaan/data/tmp/wm/BAOku56.csv')
print df.shape
#print df2.shape
df

In [ ]:
from ochre.utils import merge_wordmappings

df = merge_wordmappings(df, df2)
print df.shape

In [ ]:
df

In [ ]:
df = df.fillna('@@@')

In [ ]:
df[df['gs'] != df['original-ocr']].shape

In [ ]:
def level1_error_classification(row):
    if row['original-ocr'] == row['gs'] and row['original-ocr'] == row['corrected-ocr']:
        return 'correctly unchanged'
    if row['original-ocr'] == row['gs'] and row['corrected-ocr'] != row['gs']:
        return 'added mistake'
    
    if row['original-ocr'] != row['gs'] and row['gs'] == row['corrected-ocr']:
        return 'correctly corrected'
    if row['original-ocr'] != row['gs'] and row['original-ocr'] == row['corrected-ocr']:
        return 'uncorrected mistake'
    
    if row['original-ocr'] != row['gs'] and row['gs'] != row['corrected-ocr'] and row['original-ocr'] != row['corrected-ocr']:
        return 'changed mistake'
    
    return 'something else'

df['level1'] = df.apply(level1_error_classification, axis=1)
df

In [ ]:
overview = {}
grouped = df.groupby('level1')
for gr in grouped:
    print gr[0], gr[1].shape
    overview[gr[0]] = gr[1].shape[0]
overview

In [ ]:
overview['total words'] = sum(overview.values())

In [ ]:
idx = ['total words', 'correctly unchanged', 'correctly corrected', 'uncorrected mistake', 'changed mistake', 'added mistake']

In [ ]:
ov = pd.DataFrame.from_dict(overview, orient='index')
ov = ov.reindex(index=idx)
ov.columns = ['# words']
ov

In [ ]:
ov = ov.iloc[::-1]

In [ ]:
ov.plot.barh(figsize=(15,8), legend=False)

In [ ]:
grouped.get_group('changed mistake')

In [ ]:
grouped.get_group('uncorrected mistake')

In [ ]:
grouped.get_group('added mistake')

In [ ]:
errors = df[df['gs'] != df['corrected-ocr']]
errors = errors.fillna(u'@@@')
print errors.shape
#errors

In [ ]:
errors.columns = [u'gs', u'ocr', u'original-ocr', u'gs2', u'level1']

In [ ]:
import codecs

with codecs.open('/home/jvdzwaan/Desktop/BWNT05_3.0.1/BWNT05_v003.txt', 'rb', 'latin1') as f:
    lines = f.readlines()

terms = []
    
for line in lines:
    entry = line.split(';')
    # lexicon service needs lower case input
    term = entry[3].lower()
    term = term.replace('"', '')
    terms.append(term)

print len(terms)
terms = set(terms)

In [ ]:
from ochre.ocrerrors import categorize_errors
errors = df.query('level1=="correctly corrected"')
errors = errors.fillna(u'@@@')
print errors.shape
errors
errors = categorize_errors(errors, terms, gs_name='gs', ocr_name='original-ocr')

In [ ]:
errors

In [ ]:
errors.query('error_type=="multiple"')

In [ ]:
grouped = errors.groupby(['error_type', 'level1'])

In [ ]:
grouped.count().plot(kind='barh', legend=False)

In [ ]:
cor = {}
for names, data in grouped:
    print names, data.shape[0]
    cor[names[0]] = data.shape[0]

In [ ]:
cor

In [ ]:
corr = pd.DataFrame.from_dict(cor, orient='index')
corr = corr.reindex(index=['hyphenation', 'hyphenation2', 'punctuation', 'missing_punctuation', 
                         'whitespace', 'real_word', 'other'])
corr.columns = ['# words']
corr

In [ ]:
corr.plot.barh()

In [ ]:
errors = df.query('level1=="changed mistake" or level1=="added mistake" or level1=="uncorrected mistake"')
errors = errors.fillna(u'@@@')
print errors.shape
errors = categorize_errors(errors, terms, gs_name='gs', ocr_name='corrected-ocr')

In [ ]:
errors

In [ ]:
grouped = errors.groupby(['error_type', 'level1'])
cor2 = {}
for names, data in grouped:
    print names, data.shape[0]
    if not names[1] in cor2.keys():
        cor2[names[1]] = {}
    cor2[names[1]][names[0]] = data.shape[0]
cor2['correctly corrected'] = cor

In [ ]:
cor2

In [ ]:
res = pd.DataFrame(cor2)
res = res.fillna(0)
res = res.T
res

In [ ]:
error_order = ['added mistake', 'changed mistake', 'uncorrected mistake', 'correctly corrected']
res[['hyphenation', 'hyphenation2', 'punctuation', 'missing_punctuation',
     'missing_word', 'number', 'whitespace', 'accent', 'real_word', 'other']].loc[error_order].plot.barh(figsize=(15,8), stacked=True)

In [ ]:
errors.query('level1 == "uncorrected mistake" and error_type=="other"')

In [ ]:
errors.query('level1 == "added mistake" and error_type=="real_word"').loc[[55, 75, 262, 88, 62]]

In [ ]:
from ochre.ocrerrors import categorize_errors
corrected = df.query('level1 == "correctly corrected"')

c = categorize_errors(corrected, terms, gs_name='gs', ocr_name='original-ocr')

In [ ]:
c[['hyphenation', 'hyphenation2', 'case', 'punctuation', 'missing_punctuation', 'missing_word', 'number', 'whitespace',
 'accent', 'real_word', 'other']].sum().plot.barh(stacked=True, figsize=(15,7))

In [ ]:
c[['hyphenation', 'hyphenation2', 'case', 'punctuation', 'missing_punctuation', 'missing_word', 'number', 'whitespace',
 'accent', 'real_word', 'other']].sum()

In [ ]:
c.query('other==True')

In [ ]:
# hoeveel verfouterd? ocr != gs en original-ocr == gs
errors = df[df['gs'] != df['ocr']]
errors = errors[errors['original-ocr'] == errors['gs']]
errors = errors.fillna(u'@@@')
print errors.shape
errors

In [ ]:
for err_name, err_function in err_types.items():
    if err_name == 'real_word':
        e[err_name] = num_errors(errors, err_function, args=(terms,))
        errors = remove_errors(errors, err_function, args=(terms,))
    else:
        e[err_name] = num_errors(errors, err_function)
        errors = remove_errors(errors, err_function)
e['other'] = errors.shape[0]

In [ ]:
print e

In [ ]:
result = pd.DataFrame(e.items(), columns=['name','value'])
result = result.set_index('name')
result.columns = ['number']
result.plot.bar(y='number')